In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
import time as time


In [10]:
#import data
X_test = pd.read_csv('X_test_stratified.csv')
X_train = pd.read_csv('X_train_stratified.csv')
y_test = pd.read_csv('y_test_stratified.csv')
y_train = pd.read_csv('y_train_stratified.csv')


## Random Forest

### INITAL

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Create a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model using the training sets
rf.fit(X_train, y_train.values.ravel())

# Predict the response for the test dataset
y_pred = rf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", accuracy_score(y_test, y_pred))

# Detailed report of classification done by the model
print(classification_report(y_test, y_pred))

Accuracy: 0.6795918367346939
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         4
           4       0.70      0.21      0.33        33
           5       0.71      0.66      0.69       291
           6       0.64      0.82      0.72       440
           7       0.72      0.49      0.58       176
           8       0.95      0.54      0.69        35
           9       0.00      0.00      0.00         1

    accuracy                           0.68       980
   macro avg       0.53      0.39      0.43       980
weighted avg       0.69      0.68      0.67       980



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Tuning


In the parameter grid, I've added 'class_weight' to handle the class imbalance. It adjusts the cost function to give more weight to under-represented classes and less weight to over-represented ones. 'balanced' and 'balanced_subsample' automatically adjust weights inversely proportional to class frequencies. The 'None' option is added to compare the performance with and without class weighting.

In [15]:
### Tuning

from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'class_weight' : ['balanced', 'balanced_subsample', None]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=2, scoring='accuracy')

# Fit the GridSearch to our training data
grid_search.fit(X_train, y_train.values.ravel())

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Create a new Random Forest classifier with the best parameters
rf_best = RandomForestClassifier(**grid_search.best_params_, random_state=42)

# Fit the model on the training data
rf_best.fit(X_train, y_train.values.ravel())

# Predict the labels of the test set
y_pred = rf_best.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

# Print the confusion matrix
print(confusion_matrix(y_test, y_pred))


Best parameters:  {'class_weight': None, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best score:  0.6360387953037264
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         4
           4       0.64      0.21      0.32        33
           5       0.71      0.67      0.69       291
           6       0.64      0.80      0.71       440
           7       0.69      0.48      0.57       176
           8       1.00      0.57      0.73        35
           9       0.00      0.00      0.00         1

    accuracy                           0.67       980
   macro avg       0.52      0.39      0.43       980
weighted avg       0.68      0.67      0.66       980

[[  0   0   1   3   0   0   0]
 [  0   7  17   9   0   0   0]
 [  0   3 194  93   1   0   0]
 [  0   1  58 352  29   0   0]
 [  0   0   4  88  84   0   0]
 [  0   0   1   8   6  20   0]
 [  0   0   0   0   1   0   0]]


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


- Class 6: Similar to class 5, the precision and recall are around 0.60. This class seems to be the best predicted by the model.
- The accuracy of the model is 0.56 or 56%, which is the proportion of true results (both true positives and true negatives) in the population.
- the model is performing decently for classes 5, 6, and 7 but poorly for the other classes.

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Create a new Random Forest classifier with the best parameters
rf_best = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=20, min_samples_leaf=1,
                                 min_samples_split=2, class_weight=None, random_state=42)

# Fit the model on the training data
rf_best.fit(X_train, y_train.values.ravel())

# Predict the labels of the test set
y_pred = rf_best.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

# Print the confusion matrix
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           3       0.00      0.00      0.00         4
           4       0.64      0.21      0.32        33
           5       0.71      0.67      0.69       291
           6       0.64      0.80      0.71       440
           7       0.69      0.48      0.57       176
           8       1.00      0.57      0.73        35
           9       0.00      0.00      0.00         1

    accuracy                           0.67       980
   macro avg       0.52      0.39      0.43       980
weighted avg       0.68      0.67      0.66       980

[[  0   0   1   3   0   0   0]
 [  0   7  17   9   0   0   0]
 [  0   3 194  93   1   0   0]
 [  0   1  58 352  29   0   0]
 [  0   0   4  88  84   0   0]
 [  0   0   1   8   6  20   0]
 [  0   0   0   0   1   0   0]]


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#comapre to base model
from sklearn.metrics import classification_report, confusion_matrix

#create base model
rf_base = RandomForestClassifier(n_estimators=100, random_state=42)

#fit base model
rf_base.fit(X_train, y_train.values.ravel())

#predict base model
y_pred = rf_base.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           3       0.00      0.00      0.00         4
           4       0.70      0.21      0.33        33
           5       0.71      0.66      0.69       291
           6       0.64      0.82      0.72       440
           7       0.72      0.49      0.58       176
           8       0.95      0.54      0.69        35
           9       0.00      0.00      0.00         1

    accuracy                           0.68       980
   macro avg       0.53      0.39      0.43       980
weighted avg       0.69      0.68      0.67       980



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
# Create a Random Forest Classifier with bootstrap class weighting
rf = RandomForestClassifier(class_weight="balanced_subsample", random_state=1)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the test data
y_pred = rf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("recall:", recall_score(y_test, y_pred, average='weighted'))

/var/folders/9x/2dgcd7cn3894j3wpmddp46940000gn/T/ipykernel_16405/2944454208.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


Accuracy: 0.6663265306122449
recall: 0.6663265306122449


Accuracy: 0.22448979591836735


/var/folders/9x/2dgcd7cn3894j3wpmddp46940000gn/T/ipykernel_16405/2656044500.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train_resampled, y_train_resampled)
